# Structural linguistics

Check the current venv and installed packages:

In [1]:
import sys
print(sys.path)

['C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\python36.zip', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\DLLs', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\lib', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp', '', 'C:\\Users\\ssotn\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\lib\\site-packages', 'C:\\Users\\ssotn\\Anaconda3\\envs\\nlp\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\ssotn\\.ipython']


In [2]:
import nltk
import spacy
#!python -m spacy download en_core_web_sm

nlp_en = spacy.load('en_core_web_lg')

#### 1. Побудуйте ланцюжок походження слів за зразком:

```
оженитися: (о + (женити + ся))
```

1. атиповий - а + (тип + овий)
2. мореплавання - море + (плав + ання)
3. оподаткувати - о + ((подат + кува) + ти)
4. перевтілитися - ((пере + (в + тіл)) + ити) + ся
5. схилившись - (((с + хил) + ив) + ши) + сь
6. підсніжник - (під + сніж) + ник

#### 2. Перевірте роботу [SnowballStem](http://snowballstem.org/) для спільнокореневих слів. Напишіть ваші спостереження.

1. truth, truthful, truthfulness, countertruth, untruthful, truthology
2. flaw, flaws, flawed, flawless, flawlessness, flawlessly, 
3. лес, лесной, лесник, лесничий, лесничество, пролесье
4. окно, окошко, подоконник, оконный, окнище

In [3]:
from nltk.stem.snowball import SnowballStemmer
print(" ".join(SnowballStemmer.languages))

arabic danish dutch english finnish french german hungarian italian norwegian porter portuguese romanian russian spanish swedish


In [4]:
stemmer_en = SnowballStemmer('english')
stemmer_ru = SnowballStemmer('russian')

In [5]:
words = [
    (stemmer_en, ['truth', 'truthful', 'truthfulness', 'countertruth', 'untruthful', 'truthology']),
    (stemmer_en, ['flaw', 'flaws', 'flawed', 'flawless', 'flawlessness', 'flawlessly']),
    (stemmer_ru, ['лес', 'лесной', 'лесник', 'лесничий', 'лесничество', 'пролесье']),
    (stemmer_ru, ['окно', 'окошко', 'подоконник', 'оконный', 'окнище']),
]

for stemmer, group in words:
    print(f"Group {group[0]}:{str(stemmer.stemmer)}")
    for w in group:
        print(f"  {w} -> {stemmer.stem(w)}")

Group truth:<EnglishStemmer>
  truth -> truth
  truthful -> truth
  truthfulness -> truth
  countertruth -> countertruth
  untruthful -> untruth
  truthology -> trutholog
Group flaw:<EnglishStemmer>
  flaw -> flaw
  flaws -> flaw
  flawed -> flaw
  flawless -> flawless
  flawlessness -> flawless
  flawlessly -> flawless
Group лес:<RussianStemmer>
  лес -> лес
  лесной -> лесн
  лесник -> лесник
  лесничий -> леснич
  лесничество -> лесничеств
  пролесье -> пролес
Group окно:<RussianStemmer>
  окно -> окн
  окошко -> окошк
  подоконник -> подоконник
  оконный -> окон
  окнище -> окнищ


Як і очікувалося, стем, який вертає стемер, часто не є валідним словом людської мови (особливо для російської). Також, погано опрацьовується заміна букв в середені слів.
При цьому всьому, стемери досить просто робити для нових мов (або портувати на інші мови програмування), вони потребують малих ресурсів пам'яті. Також, до них можна додавати свої правила, щоб обробити найбільш поширені винятки. То ж їх можна досить успішно використовувати, коли нема повноцінного морфологічного движка.

#### 3. Визначте частину мови виділеного слова і зв'язок до його батька (за зразком):

```
{I} like turtles.: pronoun, nsubj(like, I)  
I {like} turtles.: verb, root(ROOT, like)  
I like {turtles}.: noun, dobj(like, turtles)
```

1. We can {but} hope that everything will be fine.
2. It's sad {but} true.
3. Jack brings nothing {but} trouble.
4. {As} we were talking, I realised how to solve the issue.
5. This hot dog isn't {as} big as usual.
6. This hot dog isn't as big {as} usual.
7. This hot dog isn't as big {as} I expected.
8. I work {as} a teacher.
9. Let's do it this {way}!
10. This is {way} too much!
11. The prices are going {down}.
12. Someone pushed him and he fell {down} the stairs.
13. I’ve been feeling rather {down} lately.
14. It's not easy to {down} a cup of coffee in one gulp.
15. Bring a {down} jacket and a pair of gloves, and you'll be fine.

In [6]:
sents = [
    "We can {but} hope that everything will be fine.",
    "It's sad {but} true.",
    "Jack brings nothing {but} trouble.",
    "{As} we were talking, I realised how to solve the issue.",
    "This hot dog isn't {as} big as usual.",
    "This hot dog isn't as big {as} usual.",
    "This hot dog isn't as big {as} I expected.",
    "I work {as} a teacher.",
    "Let's do it this {way}!",
    "This is {way} too much!",
    "The prices are going {down}.",
    "Someone pushed him and he fell {down} the stairs.",
    "I’ve been feeling rather {down} lately.",
    "It's not easy to {down} a cup of coffee in one gulp.",
    "Bring a {down} jacket and a pair of gloves, and you'll be fine.",
]

examples = [(s.replace('}', '').replace('{', ''), s.find('{'), s)  
            for s in sents]

print(examples[:2])

[('We can but hope that everything will be fine.', 7, 'We can {but} hope that everything will be fine.'), ("It's sad but true.", 9, "It's sad {but} true.")]


In [7]:
for i, (sent, w_pos, original) in enumerate(examples):
    doc = nlp_en(sent)
    for word in doc:
        if word.idx != w_pos: 
            continue
        
        print(f"{i+1}. {original}: {word.pos_}, {word.dep_}({word.head.text}, {word})")
        break

1. We can {but} hope that everything will be fine.: CCONJ, cc(can, but)
2. It's sad {but} true.: CCONJ, cc(sad, but)
3. Jack brings nothing {but} trouble.: ADP, prep(nothing, but)
4. {As} we were talking, I realised how to solve the issue.: ADP, mark(talking, As)
5. This hot dog isn't {as} big as usual.: ADV, advmod(big, as)
6. This hot dog isn't as big {as} usual.: ADP, prep(big, as)
7. This hot dog isn't as big {as} I expected.: ADP, mark(expected, as)
8. I work {as} a teacher.: ADP, prep(work, as)
9. Let's do it this {way}!: NOUN, npadvmod(do, way)
10. This is {way} too much!: ADV, advmod(much, way)
11. The prices are going {down}.: PART, prt(going, down)
12. Someone pushed him and he fell {down} the stairs.: ADP, prep(fell, down)
13. I’ve been feeling rather {down} lately.: ADV, advmod(feeling, down)
14. It's not easy to {down} a cup of coffee in one gulp.: VERB, xcomp('s, down)
15. Bring a {down} jacket and a pair of gloves, and you'll be fine.: ADJ, amod(jacket, down)


#### 4. Визначте частину мови виділеного слова і зв'язок до його батька (за зразком):

```
{Я} люблю черепашок.: займенник, nsubj(люблю, Я)  
Я {люблю} черепашок.: дієслово, root(ROOT, люблю)  
Я люблю {черепашок}.: іменник, dobj(люблю, черепашок)  
```

1. Рада міністрів Європейського союзу затвердила угоду про спрощений порядок видачі {віз} для України.: іменник, DOBJ(видачі, віз)
2. Батько Себастьяна {віз} на санях їх театральний гурт до Львова.: дієслово, ROOT(root, віз)
3. А ще дивний елемент інтер’єру – {віз} із продукцією одного з херсонських виробників.: іменник, APPOS(елемент, віз)
4. У цю мить {повз} Євгена пролетів останній вагон товарняка.: прийменник, PREP(пролетів, повз)
5. Кліпнув очима і побачив малого песика, який саме пробігав {повз} у бік села.: прислівник, ADVMOD(пробігав, повз)
6. Степанко перестав кричати, тільки ламкий стогін {повз} йому із грудей.: дієслово, CCOMP(кричати, повз)
7. Часу не {гай} – декларацію подай!: дієслово, ROOT(root, гай)
8. І коляд заспівали, і {гай} врятували.: іменник, DOBJ(врятували, гай)
9. {Гай}, чи ви забулися, братове?: вигук, INTJ(забулися, Гай)
10. Ось присіла на {край} ліжка.: іменник, POSS(ліжка, край)
11. Поставив ту кузню не {край} дороги, як було заведено, а на Красній горі, біля Прадуба.: прийменник, PREP(поставив, край)
12. Розповідаючи про передній {край} лінґвістики, фон Лібіх, як завжди, мислив широко і глобально.: іменник, POBJ(про, край)
13. Не {край} мені серце.: дієслово, root(ROOT, край)
14. І {щойно} тоді додаємо до борщу смажену цибулю.: частка, ADVMOD(тоді, щойно)
15. Бо {щойно} я задрімав, віддавши тіло подушці з периною, як мене розбудив поштовх у бік.: прислівник, ADVMOD(задрімав, щойно)

#### 5. Виберіть одне речення зі списку та побудуйте для нього дерево залежностей та дерево складників:

(Можна намалювати дерева в графічному редакторі або на папері і докріпити картинку до домашнього завдання. Назви складників, частин мови та залежностей можна не писати. Головне - це структура.)

1. Пригадую, уже згодом, коли я відбував свій термін у таборі № 36 у Кучино Пермської області, я отримав від Михасі листівку з жартівливим описом того, як Київ готується до святкування свого 1500-ліття.
2. 6C приземляється на плече, перекочуючись, пролітає метрів п’ятдесят і витягується на снігу за кілька кроків від забризканої палаючими уламками посадкової смуги.
3. Дівчина стояла там, де й була, і намагалася привести до ладу скуйовджене волосся, вкрай розлючена тим, що це побачили водії, які чекали на переїзді.

In [8]:
from spacy import displacy

tokens = [
        ('Пригадую', ''), # root
        ('уже', 'згодом,'),
        ('згодом,', 'отримав'),
        ('коли', 'відбував'),
        ('я', 'відбував'),
        ('відбував', 'отримав'),
        ('свій', 'термін'),
        ('термін', 'відбував'),
        ('у', 'таборі'),
        ('таборі', 'відбував'),
        ('№', '36'),
        ('36', 'відбував'),
        ('_у', 'області,'),
        ('Кучино', 'Пермської'),
        ('Пермської', 'області,'),
        ('області,', 'відбував'),
        ('_я', 'отримав'),
        ('отримав', 'Пригадую'),
        ('від', 'Михасі'),
        ('Михасі', 'отримав'),
        ('листівку', 'отримав'),
        ('з', 'описом'),
        ('жартівливим', 'описом'),
        ('описом', 'листівку'),
        ('того,', 'описом'),
        ('як', 'готується'),
        ('Київ', 'готується'),
        ('готується', 'того,'),
        ('до', 'святкування'),
        ('святкування', 'готується'),
        ('свого', '1500-ліття.'),
        ('1500-ліття.', 'святкування'),
]

idx = dict({t[0]:i for i, t in enumerate(tokens)})
# print(idx['до'])

ex = [{
    'words':[
    ],
    'arcs': [        
    ]
}]

arcs = ex[0]['arcs']
words = ex[0]['words']
for i, t in enumerate(tokens):
    words.append({
        'text':t[0].replace('_',''),
        'tag':'ROOT' if t[1]=='' else ''
    })
    to = t[1]
    if to!='':
        to_idx = idx[to]
        arcs.append({'start':i, 'end':to_idx, 'label':'', 'dir':'left'})        

svg = displacy.render(ex, style='dep', manual=True, options={'compact':False})
open('sentence.svg', 'w', encoding='utf-8').write(svg)

24103

Dependency tree stored in file **[sentence.svg](sentence.svg)**
![sentence.svg](sentence.svg)

#### 6. Виберіть одне cлово зі списку та побудуйте лексико-семантичні зв'язки до трьох різних значень цього слова. Фактично, потрібно побудувати WordNet-подібні вузли. Значення слів можна перевірити у [СУМі](http://sum.in.ua/) та [Словниках України](http://lcorp.ulif.org.ua/dictua/).

Слова на вибір: вік, стіна, добро, серце, центр, куля, міст, ланцюг, бік, дух.

Міст
```
1. Споруда для переїзду або переходу через річку, залізницю, автомагістраль, яр і т.ін.
гіпероніми: споруда
сестри: перехід, шляхопровід, перелаз
гіпоніми: автомобільний міст, мотузковий міст, залізничний міст
мероніми: проліт, опора, покриття

2. Положення тіла з вигнутою догори грудною кліткою і з упором на долоні й п'яти.
гіпероніми: поза
сестри: шпагат, стійка
гіпоніми: борцівський міст, гімнастичний міст
мероніми: 

3. Частина шасі автомашини, трактора.
гіпероніми: автомобіль
сестри: рама, двигун, кузов
гіпоніми: розрізний міст, нерозрізний міст
мероніми: вісь, колесо, діференціал
```